In [1]:
#import pandas and sqlite
import pandas as pd
import sqlite3 as sl

In [2]:
#connecct to database object
conn = sl.connect('NYT_covid.db')
c = conn.cursor()

In [3]:
#update counties table with most recent data
counties = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv', index_col=0, encoding='latin-1')

c.execute('DROP TABLE IF EXISTS counties')
c.execute('CREATE TABLE counties (date datetime, county str, state str, fips str, cases int, deaths int, confirmed_cases int, confirmed_deaths int, probable_cases int, probable_deaths int)')
conn.commit()

counties.to_sql('counties',conn, if_exists='replace')

In [4]:
#confirm tables present
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print (name[0])

deaths
mask_use
us
election
states
regions
counties


In [5]:
df = pd.read_sql('''select * from regions''',conn)
df.columns

Index(['State', 'census_bureau_region', 'census_bureau_division',
       'standard_federal_region', 'circiut_court_of_appeals',
       'bureau_economic_analysis'],
      dtype='object')

In [6]:
df = pd.read_sql('''select * from counties''',conn)
df.columns

Index(['date', 'county', 'state', 'fips', 'cases', 'deaths'], dtype='object')

In [9]:
df = pd.read_sql('''WITH cte as (SELECT date, 
                                  r.bureau_economic_analysis as 'bea_region', c.state, c.county, 
                                  cases - LAG (cases,1) OVER (PARTITION BY fips ORDER BY date) as 'new_cases',
                                  cases as 'cumulative_cases'
                    FROM     counties c
                    JOIN     regions r on c.state = r.state 
                             AND r.bureau_economic_analysis = 'Great Lakes'
                    ORDER BY c.state, county)
                    
SELECT date,
       bea_region, 
       state, 
       county, 
       new_cases, 
       cumulative_cases,
       AVG(new_cases) OVER (PARTITION BY state, county ORDER BY date ASC rows 6 PRECEDING) as '7-day_avg_new'
FROM cte
ORDER BY state, county, date''',conn)
df

,date,bea_region,state,county,new_cases,cumulative_cases,7-day_avg_new
0,2020-03-20,Great Lakes,Illinois,Adams,NaN,1,NaN
1,2020-03-21,Great Lakes,Illinois,Adams,0.0,1,0.000000
2,2020-03-22,Great Lakes,Illinois,Adams,0.0,1,0.000000
3,2020-03-23,Great Lakes,Illinois,Adams,0.0,1,0.000000
4,2020-03-24,Great Lakes,Illinois,Adams,0.0,1,0.000000
...,...,...,...,...,...,...,...
104252,2020-11-15,Great Lakes,Wisconsin,Wood,29.0,3291,99.571429
104253,2020-11-16,Great Lakes,Wisconsin,Wood,29.0,3320,90.000000
104254,2020-11-17,Great Lakes,Wisconsin,Wood,86.0,3406,84.857143
104255,2020-11-18,Great Lakes,Wisconsin,Wood,206.0,3612,105.857143
